In [16]:
# Importing packages
import requests  
import pandas as pd
import sklearn
import nltk
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction import text
from sklearn.metrics import ConfusionMatrixDisplay
from nltk.stem import WordNetLemmatizer
from sklearn import svm
import time

In [17]:
#Importing csv as data frame
mtgDF = pd.read_csv("MTGData.csv")

In [18]:
mtgDF

,Name,Text,Flavor,Color,Cost,Type,Rarity
0,+2 Mace,Equipped creature gets +2/+2 Equip {3} {3}: At...,The weight of this magic weapon falls heavy on...,['W'],{1}{W},Artifact — Equipment,common
1,A-Ancestral Katana,Whenever a Samurai or Warrior you control atta...,NaN,['W'],{1}{W},Artifact — Equipment,common
2,Abbey Gargoyles,Flying protection from red,Though Serra has not been seen for twenty year...,['W'],{2}{W}{W}{W},Creature — Gargoyle,uncommon
3,Abbey Griffin,Flying vigilance,The darkness crawls with vampires and ghouls b...,['W'],{3}{W},Creature — Griffin,common
4,Abbey Matron,{W} {T}: Abbey Matron gets +0/+3 until end of ...,The Matrons are kindly souls but don t ask one...,['W'],{2}{W},Creature — Human Cleric,common
...,...,...,...,...,...,...,...
19245,Wolf Strike,Target creature you control gets +2/+0 until e...,Tovolar s howlpack had scattered but the Dire ...,['G'],{2}{G},Instant,common
19246,Wolfwillow Haven,Enchant land Whenever enchanted land is tapped...,NaN,['G'],{1}{G},Enchantment — Aura,uncommon
19247,Wolverine Pack,Rampage 2 Whenever this creature becomes block...,Give them great meals of beef and iron and ste...,['G'],{2}{G}{G},Creature — Wolverine,uncommon
19248,Wolverine Riders,At the beginning of each upkeep create a 1/1 g...,We ll break their lines The rest of you follow,['G'],{4}{G}{G},Creature — Elf Warrior,rare


In [19]:
#Removing all rows with any NA values from the data
mtgDF = mtgDF[mtgDF["Color"].notna()]
mtgDF = mtgDF[mtgDF["Name"].notna()]
mtgDF = mtgDF[mtgDF["Text"].notna()]
mtgDF = mtgDF[mtgDF["Flavor"].notna()]
mtgDF = mtgDF[mtgDF["Cost"].notna()]
mtgDF = mtgDF[mtgDF["Type"].notna()]
mtgDF = mtgDF[mtgDF["Rarity"].notna()]
mtgDF = mtgDF.reset_index()
mtgDF = mtgDF.drop(columns = ["index"])

In [20]:
mtgDF

,Name,Text,Flavor,Color,Cost,Type,Rarity
0,+2 Mace,Equipped creature gets +2/+2 Equip {3} {3}: At...,The weight of this magic weapon falls heavy on...,['W'],{1}{W},Artifact — Equipment,common
1,Abbey Gargoyles,Flying protection from red,Though Serra has not been seen for twenty year...,['W'],{2}{W}{W}{W},Creature — Gargoyle,uncommon
2,Abbey Griffin,Flying vigilance,The darkness crawls with vampires and ghouls b...,['W'],{3}{W},Creature — Griffin,common
3,Abbey Matron,{W} {T}: Abbey Matron gets +0/+3 until end of ...,The Matrons are kindly souls but don t ask one...,['W'],{2}{W},Creature — Human Cleric,common
4,Abeyance,Until end of turn target player can t cast ins...,I m too modest a wizard to reveal the full ext...,['W'],{1}{W},Instant,rare
...,...,...,...,...,...,...,...
12039,Wolf's Quarry,Create three 1/1 green Boar creature tokens wi...,The monster was gaining on them Twice it had f...,['G'],{4}{G}{G},Sorcery,common
12040,Wolf Strike,Target creature you control gets +2/+0 until e...,Tovolar s howlpack had scattered but the Dire ...,['G'],{2}{G},Instant,common
12041,Wolverine Pack,Rampage 2 Whenever this creature becomes block...,Give them great meals of beef and iron and ste...,['G'],{2}{G}{G},Creature — Wolverine,uncommon
12042,Wolverine Riders,At the beginning of each upkeep create a 1/1 g...,We ll break their lines The rest of you follow,['G'],{4}{G}{G},Creature — Elf Warrior,rare


In [21]:
#Renaming the card colors to regular strings
for i in range(len(mtgDF)):
    if mtgDF.iloc[i]["Color"] == "['W']":
         mtgDF.iloc[i]["Color"] = "White"
    elif mtgDF.iloc[i]["Color"] == "['G']":
         mtgDF.iloc[i]["Color"] = "Green"
    elif mtgDF.iloc[i]["Color"] == "['R']":
         mtgDF.iloc[i]["Color"] = "Red"
    elif mtgDF.iloc[i]["Color"] == "['U']":
         mtgDF.iloc[i]["Color"] = "Blue"
    elif mtgDF.iloc[i]["Color"] == "['B']":
         mtgDF.iloc[i]["Color"] = "Black"
    else:
        mtgDF.iloc[i]["Color"] = "NA"

In [22]:
#Checking for any missing values
mtgDF.isna().sum()

Name      0
Text      0
Flavor    0
Color     0
Cost      0
Type      0
Rarity    0
dtype: int64

In [23]:
#Writing the new data frame to a csv file
mtgDF.to_csv("MTG_Cards")

In [24]:
mtgDF

,Name,Text,Flavor,Color,Cost,Type,Rarity
0,+2 Mace,Equipped creature gets +2/+2 Equip {3} {3}: At...,The weight of this magic weapon falls heavy on...,White,{1}{W},Artifact — Equipment,common
1,Abbey Gargoyles,Flying protection from red,Though Serra has not been seen for twenty year...,White,{2}{W}{W}{W},Creature — Gargoyle,uncommon
2,Abbey Griffin,Flying vigilance,The darkness crawls with vampires and ghouls b...,White,{3}{W},Creature — Griffin,common
3,Abbey Matron,{W} {T}: Abbey Matron gets +0/+3 until end of ...,The Matrons are kindly souls but don t ask one...,White,{2}{W},Creature — Human Cleric,common
4,Abeyance,Until end of turn target player can t cast ins...,I m too modest a wizard to reveal the full ext...,White,{1}{W},Instant,rare
...,...,...,...,...,...,...,...
12039,Wolf's Quarry,Create three 1/1 green Boar creature tokens wi...,The monster was gaining on them Twice it had f...,Green,{4}{G}{G},Sorcery,common
12040,Wolf Strike,Target creature you control gets +2/+0 until e...,Tovolar s howlpack had scattered but the Dire ...,Green,{2}{G},Instant,common
12041,Wolverine Pack,Rampage 2 Whenever this creature becomes block...,Give them great meals of beef and iron and ste...,Green,{2}{G}{G},Creature — Wolverine,uncommon
12042,Wolverine Riders,At the beginning of each upkeep create a 1/1 g...,We ll break their lines The rest of you follow,Green,{4}{G}{G},Creature — Elf Warrior,rare
